*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
label

In [ ]:
dataset.isnull().sum()
#seems no any missing value

In [ ]:
dataset['bmi'].max()
dataset['bmi'].min()
#maximum age is 64 and minimum age is 18
#maximum bmi is 53.1 and minimum bmi is 16

In [ ]:
#age numberization
'''
[18-19] -> 1 
[20-26] -> 2 
[27-33] -> 3
[34-40] -> 4
[41-47] -> 5
[48-54] -> 6
[55-64] -> 7
'''

dataset.loc[ dataset['age'] <= 19, 'age'] 					               = 1
dataset.loc[(dataset['age'] > 19) & (dataset['age'] <= 26), 'age'] = 2
dataset.loc[(dataset['age'] > 27) & (dataset['age'] <= 33), 'age'] = 3
dataset.loc[(dataset['age'] > 33) & (dataset['age'] <= 40), 'age'] = 4
dataset.loc[(dataset['age'] > 40) & (dataset['age'] <= 47), 'age'] = 5
dataset.loc[(dataset['age'] > 47) & (dataset['age'] <= 54), 'age'] = 6
dataset.loc[(dataset['age'] > 54) & (dataset['age'] <= 64), 'age'] = 7

In [ ]:
dataset.loc[ dataset['smoker'] == "yes", 'smoker'] = 0
dataset.loc[dataset['smoker'] == "no", 'smoker']  = 1

In [ ]:
dataset.loc[ dataset['region'] == "northeast", 'region'] = 0
dataset.loc[dataset['region'] == "northwest", 'region']  = 1
dataset.loc[dataset['region'] == "southeast", 'region']  = 2
dataset.loc[dataset['region'] == "southwest", 'region']  = 3

In [ ]:
dataset.loc[ dataset['sex'] == "female", 'sex'] = 0
dataset.loc[dataset['sex'] == "male", 'sex']  = 1

In [ ]:
#we shall make max-min standardize for bmi

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

dataset[["bmi"]] = scaler.fit_transform(dataset[["bmi"]])


In [ ]:
label = dataset[["expenses"]]
dataset.drop(["expenses"],axis=1,inplace=True)

In [ ]:
dataset

In [ ]:
dataset.dtypes

In [ ]:
dataset = np.asarray(dataset).astype(np.float32)

In [ ]:
from sklearn.model_selection import train_test_split
train_dataset, test_dataset,train_labels,test_labels = train_test_split(dataset,label,test_size=0.33, random_state=0)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(128, input_shape= (6 , ), activation='relu', name='dense_1'))
model.add(Dense(64, activation='relu', name='dense_2'))
model.add(Dense(8, activation='relu', name='dense_3'))
model.add(Dense(1, activation='linear', name='dense_output'))
model.compile(optimizer='adam', loss='mse', metrics=['mae', "mse"])
model.summary()


In [ ]:
x_train.dtypes

In [ ]:
train_labels.dtypes

In [ ]:
history = model.fit( train_dataset, train_labels.values, epochs=50, validation_split=0.05)

In [ ]:
dataset['bmi'].hist(bins = 40)
#seems age = 19 and 20 have much more samples than other ages.

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
